In [1]:
import wfdb
import csv
import pandas as pd
import numpy as np
import os

In [2]:
import os
import csv
import pandas as pd
import wfdb  # Make sure wfdb is installed

output_dir = "data_creation"
os.makedirs(output_dir, exist_ok=True)

# Patient numbers
patient_numbers = [
    "100", "101", "102", "103", "104", "105", "106", "107", "108", "109",
    "111", "112", "113", "114", "115", "116", "117", "118", "119", "121",
    "122", "123", "124", "200", "201", "202", "203", "205", "207", "208",
    "209", "210", "212", "213", "214", "215", "217", "219", "220", "221",
    "222", "223", "228", "230", "231", "232", "233", "234"
]

# N = normal 
# S = supra-ventricular premature
# V = ventricular escape
# F = fusion of ventricular and normal
# Q = unclassified heartbeats
symbol_to_category = {
    'N': 'N', '.': 'N', 'L': 'N', 'R': 'N', 'e': 'N', 'j': 'N',
    'a': 'S', 'A': 'S', 'J': 'S', 'S': 'S',
    'V': 'V', 'E': 'V',
    'F': 'F',
    '/': 'Q', 'f': 'Q', 'Q': 'Q'
}

for patient_number in patient_numbers:
    try:
        # ECG data
        path_to_record = f"mit-database/{patient_number}"
        patient_record = wfdb.rdrecord(path_to_record)
        leads = patient_record.sig_name
        ecg_data = patient_record.p_signal

        # ECG CSV
        ecg_filename = f"{output_dir}/{patient_number}_ECG.csv"
        with open(ecg_filename, "w", newline='') as outfile:
            out_csv = csv.writer(outfile)
            out_csv.writerow(leads)
            for row in ecg_data:
                out_csv.writerow(row)

        # Annotations data
        annotation = wfdb.rdann(path_to_record, 'atr')
        symbols = annotation.symbol
        annotations = annotation.sample

        # Filter out symbols not in symbol_to_category
        filtered_symbols_annotations = [(sym, ann) for sym, ann in zip(symbols, annotations) if sym in symbol_to_category]
        categories = [symbol_to_category[sym] for sym, ann in filtered_symbols_annotations]
        annotations_filtered = [ann for sym, ann in filtered_symbols_annotations]

        df_annotations = pd.DataFrame({'Category': categories, 'Annotation': annotations_filtered})

        # Annotations CSV
        annotations_filename = f"{output_dir}/{patient_number}_Annotations.csv"
        df_annotations.to_csv(annotations_filename, index=False)

    except Exception as e:
        print(f"Failed to process: {patient_number}: {e}")

print("Done")

Done


In [3]:
def process_patient_data(patient_number, data_creation_dir="data_creation"):

    ecg_file_path = os.path.join(data_creation_dir, f"{patient_number}_ECG.csv")
    annotations_file_path = os.path.join(data_creation_dir, f"{patient_number}_Annotations.csv")
    
    patient_X = []
    patient_Y = []
    
    try:
        ecg_df = pd.read_csv(ecg_file_path)
        annotations_df = pd.read_csv(annotations_file_path)
    except FileNotFoundError:
        print(f"Files for patient {patient_number} not found. Skipping...")
        return [], []
    
    first_column_name = ecg_df.columns[0]

    sampling_rate = 360  # Hz
    window_size_seconds = 5  # Seconds before and after annotation
    window_size_samples = window_size_seconds * sampling_rate

    for _, row in annotations_df.iterrows():
        annotation_point = row['Annotation']
        category = row['Category']
        
        start_point = max(0, annotation_point - window_size_samples)
        end_point = min(len(ecg_df), annotation_point + window_size_samples)
        
        window_data = ecg_df.iloc[start_point:end_point][first_column_name].to_numpy()
        if len(window_data) < window_size_samples * 2:
            window_data = np.pad(window_data, (0, window_size_samples * 2 - len(window_data)), 'constant')
        
        patient_X.append(window_data)
        patient_Y.append(category)
    
    return patient_X, patient_Y

# Initialize lists to hold the entire dataset
all_X = []
all_Y = []

data_creation_dir = "data_creation"

# Process each patient
for patient_number in patient_numbers:
    patient_X, patient_Y = process_patient_data(patient_number, data_creation_dir)
    all_X.extend(patient_X)
    all_Y.extend(patient_Y)

X = np.array(all_X)
Y = np.array(all_Y)

In [4]:
X

array([[-0.145, -0.145, -0.145, ...,  0.   ,  0.   ,  0.   ],
       [-0.145, -0.145, -0.145, ...,  0.   ,  0.   ,  0.   ],
       [-0.145, -0.145, -0.145, ...,  0.   ,  0.   ,  0.   ],
       ...,
       [-0.245, -0.25 , -0.255, ...,  0.   ,  0.   ,  0.   ],
       [-0.195, -0.2  , -0.215, ...,  0.   ,  0.   ,  0.   ],
       [-0.225, -0.23 , -0.235, ...,  0.   ,  0.   ,  0.   ]])

In [5]:
Y

array(['N', 'N', 'N', ..., 'N', 'N', 'N'], dtype='<U1')

In [6]:
df_x = pd.DataFrame(X)
df_x

,0,1,2,3,4,5,6,7,8,9,...,3590,3591,3592,3593,3594,3595,3596,3597,3598,3599
0,-0.145,-0.145,-0.145,-0.145,-0.145,-0.145,-0.145,-0.145,-0.120,-0.135,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.145,-0.145,-0.145,-0.145,-0.145,-0.145,-0.145,-0.145,-0.120,-0.135,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.145,-0.145,-0.145,-0.145,-0.145,-0.145,-0.145,-0.145,-0.120,-0.135,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.145,-0.145,-0.145,-0.145,-0.145,-0.145,-0.145,-0.145,-0.120,-0.135,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.145,-0.145,-0.145,-0.145,-0.145,-0.145,-0.145,-0.145,-0.120,-0.135,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109489,-0.275,-0.285,-0.270,-0.260,-0.255,-0.260,-0.270,-0.265,-0.280,-0.270,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
109490,-0.230,-0.230,-0.235,-0.240,-0.245,-0.250,-0.255,-0.245,-0.245,-0.245,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
109491,-0.245,-0.250,-0.255,-0.260,-0.270,-0.255,-0.245,-0.255,-0.260,-0.255,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
109492,-0.195,-0.200,-0.215,-0.220,-0.230,-0.220,-0.220,-0.200,-0.195,-0.185,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
df_y = pd.DataFrame(Y, columns=['ColumnName'])
print(df_y)

       ColumnName
0               N
1               N
2               N
3               N
4               N
...           ...
109489          N
109490          N
109491          N
109492          N
109493          N

[109494 rows x 1 columns]


In [8]:
value_counts_y = df_y['ColumnName'].value_counts()
print(value_counts_y)

ColumnName
N    90631
Q     8043
V     7236
S     2781
F      803
Name: count, dtype: int64


In [9]:
df_fusionné = pd.concat([df_y, df_x], axis=1)
df_fusionné

,ColumnName,0,1,2,3,4,5,6,7,8,...,3590,3591,3592,3593,3594,3595,3596,3597,3598,3599
0,N,-0.145,-0.145,-0.145,-0.145,-0.145,-0.145,-0.145,-0.145,-0.120,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,N,-0.145,-0.145,-0.145,-0.145,-0.145,-0.145,-0.145,-0.145,-0.120,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,N,-0.145,-0.145,-0.145,-0.145,-0.145,-0.145,-0.145,-0.145,-0.120,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,N,-0.145,-0.145,-0.145,-0.145,-0.145,-0.145,-0.145,-0.145,-0.120,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,N,-0.145,-0.145,-0.145,-0.145,-0.145,-0.145,-0.145,-0.145,-0.120,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109489,N,-0.275,-0.285,-0.270,-0.260,-0.255,-0.260,-0.270,-0.265,-0.280,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
109490,N,-0.230,-0.230,-0.235,-0.240,-0.245,-0.250,-0.255,-0.245,-0.245,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
109491,N,-0.245,-0.250,-0.255,-0.260,-0.270,-0.255,-0.245,-0.255,-0.260,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
109492,N,-0.195,-0.200,-0.215,-0.220,-0.230,-0.220,-0.220,-0.200,-0.195,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# df_fusionné.to_csv('df_fusionné.csv', index=False)

In [11]:
df_fusionné_binaire = df_fusionné.copy()
df_fusionné_binaire.iloc[:, 0] = df_fusionné_binaire.iloc[:, 0].apply(lambda x: 0 if x != 'N' else 1)

In [12]:
df_fusionné_binaire

,ColumnName,0,1,2,3,4,5,6,7,8,...,3590,3591,3592,3593,3594,3595,3596,3597,3598,3599
0,1,-0.145,-0.145,-0.145,-0.145,-0.145,-0.145,-0.145,-0.145,-0.120,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,-0.145,-0.145,-0.145,-0.145,-0.145,-0.145,-0.145,-0.145,-0.120,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,-0.145,-0.145,-0.145,-0.145,-0.145,-0.145,-0.145,-0.145,-0.120,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,-0.145,-0.145,-0.145,-0.145,-0.145,-0.145,-0.145,-0.145,-0.120,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,-0.145,-0.145,-0.145,-0.145,-0.145,-0.145,-0.145,-0.145,-0.120,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109489,1,-0.275,-0.285,-0.270,-0.260,-0.255,-0.260,-0.270,-0.265,-0.280,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
109490,1,-0.230,-0.230,-0.235,-0.240,-0.245,-0.250,-0.255,-0.245,-0.245,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
109491,1,-0.245,-0.250,-0.255,-0.260,-0.270,-0.255,-0.245,-0.255,-0.260,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
109492,1,-0.195,-0.200,-0.215,-0.220,-0.230,-0.220,-0.220,-0.200,-0.195,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


1) model classifiacation si c'est normal (0) ou pas normal (1)

In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split

X = df_fusionné_binaire.iloc[:, 1:]  
y = df_fusionné_binaire.iloc[:, 0] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [14]:
print(y_train.unique())


[1 0]


In [15]:
X_test

,0,1,2,3,4,5,6,7,8,9,...,3590,3591,3592,3593,3594,3595,3596,3597,3598,3599
22653,-0.310,-0.320,-0.325,-0.325,-0.315,-0.300,-0.280,-0.270,-0.285,-0.290,...,-0.335,-0.350,-0.380,-0.375,-0.355,-0.335,-0.335,-0.360,-0.365,-0.370
53240,-0.175,-0.175,-0.155,-0.145,-0.140,-0.170,-0.175,-0.175,-0.170,-0.150,...,0.160,0.125,0.090,0.050,0.015,-0.015,-0.025,-0.050,-0.065,-0.100
466,-0.475,-0.460,-0.455,-0.465,-0.480,-0.475,-0.465,-0.455,-0.450,-0.460,...,-0.285,-0.280,-0.295,-0.310,-0.310,-0.295,-0.285,-0.290,-0.290,-0.295
6985,-0.230,-0.205,-0.180,-0.165,-0.160,-0.155,-0.150,-0.110,-0.090,-0.080,...,-0.215,-0.225,-0.245,-0.255,-0.280,-0.285,-0.290,-0.285,-0.300,-0.305
20492,-0.650,-0.635,-0.625,-0.605,-0.570,-0.545,-0.525,-0.520,-0.520,-0.535,...,-0.240,-0.250,-0.260,-0.240,-0.225,-0.235,-0.230,-0.245,-0.250,-0.235
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90923,-0.045,-0.040,-0.060,-0.040,-0.005,0.055,0.110,0.140,0.155,0.160,...,-0.085,-0.090,-0.085,-0.090,-0.100,-0.105,-0.110,-0.090,-0.070,-0.095
17190,-2.370,-2.445,-2.525,-2.605,-2.670,-2.725,-2.695,-2.600,-2.440,-2.240,...,-0.335,-0.335,-0.335,-0.320,-0.320,-0.310,-0.325,-0.310,-0.330,-0.310
16716,-2.130,-1.905,-1.690,-1.450,-1.220,-1.080,-0.985,-0.930,-0.900,-0.860,...,-0.190,-0.155,-0.160,-0.145,-0.120,-0.075,-0.065,-0.055,-0.040,-0.025
37205,-0.895,-0.880,-0.865,-0.885,-0.930,-0.985,-1.020,-1.035,-1.015,-0.945,...,-0.890,-0.925,-0.945,-0.990,-1.020,-1.045,-1.030,-1.010,-0.995,-1.005


In [16]:
X_train

,0,1,2,3,4,5,6,7,8,9,...,3590,3591,3592,3593,3594,3595,3596,3597,3598,3599
72220,0.175,0.180,0.220,0.235,0.220,0.150,0.060,0.050,0.115,0.170,...,0.130,0.105,0.025,-0.045,-0.075,-0.070,-0.060,-0.065,-0.085,-0.085
75099,-0.920,-0.900,-0.900,-0.925,-0.925,-0.950,-0.955,-0.960,-0.975,-0.985,...,1.010,1.250,1.480,1.720,1.970,2.175,2.345,2.490,2.585,2.670
35766,-1.020,-1.020,-1.000,-1.000,-1.015,-1.010,-1.025,-1.005,-1.025,-1.015,...,-0.470,-0.465,-0.470,-0.495,-0.510,-0.540,-0.565,-0.590,-0.620,-0.660
28758,-0.240,-0.210,-0.130,-0.105,-0.155,-0.230,-0.275,-0.285,-0.280,-0.260,...,-0.200,-0.175,-0.180,-0.195,-0.205,-0.210,-0.185,-0.160,-0.170,-0.190
85930,-0.840,-0.850,-0.860,-0.870,-0.855,-0.855,-0.870,-0.850,-0.840,-0.850,...,-0.820,-0.825,-0.815,-0.815,-0.820,-0.790,-0.785,-0.780,-0.775,-0.795
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54886,0.110,0.125,0.120,0.115,0.110,0.110,0.105,0.110,0.115,0.105,...,-0.040,-0.025,-0.010,0.010,-0.005,0.015,0.000,0.010,-0.010,-0.005
76820,-0.170,-0.170,-0.155,-0.135,-0.145,-0.165,-0.165,-0.165,-0.140,-0.130,...,-0.375,-0.370,-0.400,-0.420,-0.430,-0.415,-0.420,-0.430,-0.445,-0.430
103694,-0.340,-0.325,-0.335,-0.320,-0.325,-0.335,-0.335,-0.330,-0.325,-0.330,...,-0.080,-0.075,-0.090,-0.100,-0.095,-0.110,-0.110,-0.105,-0.095,-0.085
860,-0.280,-0.115,0.090,0.325,0.615,0.855,0.975,0.980,0.780,0.420,...,-0.270,-0.265,-0.265,-0.270,-0.265,-0.275,-0.280,-0.275,-0.265,-0.255


In [17]:
y_train = y_train.astype(int)
y_test = y_test.astype(int)

In [18]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

model = DecisionTreeClassifier()

param_grid = {
    'max_depth': [20, 30, 40],
    'min_samples_leaf': [5, 10, 20]
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy')

grid_search.fit(X_train, y_train)


print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best Score: {grid_search.best_score_}")

best_estimator = grid_search.best_estimator_
predictions = best_estimator.predict(X_test)

Best Parameters: {'max_depth': 40, 'min_samples_leaf': 5}
Best Score: 0.9667218448541582


In [19]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(max_depth=40, min_samples_leaf=5)
model.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=40, min_samples_leaf=5)

In [21]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, log_loss

# Prédiction sur l'ensemble de test
y_pred = model.predict(X_test)

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", conf_matrix)

# Precision
precision = precision_score(y_test, y_pred)
print(f"Precision: {precision:.2f}")

# Recall
recall = recall_score(y_test, y_pred)
print(f"Recall: {recall:.2f}")

# F1 Score
f1 = f1_score(y_test, y_pred)
print(f"F1 Score: {f1:.2f}")

# ROC-AUC Score
y_pred_proba = model.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f"ROC AUC Score: {roc_auc:.2f}")

# Log Loss
logloss = log_loss(y_test, y_pred_proba)
print(f"Log Loss: {logloss:.2f}")

Confusion Matrix:
 [[ 3321   431]
 [  286 17861]]
Precision: 0.98
Recall: 0.98
F1 Score: 0.98
ROC AUC Score: 0.94
Log Loss: 0.64


Model for the anormal heart beat

In [20]:
df_anormal = df_fusionné[df_fusionné["ColumnName"] != "N"]
df_anormal

,ColumnName,0,1,2,3,4,5,6,7,8,...,2150,2151,2152,2153,2154,2155,2156,2157,2158,2159
7,S,-0.405,-0.410,-0.405,-0.400,-0.390,-0.395,-0.410,-0.415,-0.410,...,-0.270,-0.265,-0.260,-0.275,-0.285,-0.265,-0.255,-0.250,-0.270,-0.280
230,S,-0.410,-0.410,-0.395,-0.385,-0.380,-0.390,-0.400,-0.405,-0.395,...,-0.385,-0.395,-0.390,-0.385,-0.385,-0.395,-0.385,-0.375,-0.370,-0.380
258,S,-0.100,0.085,0.335,0.600,0.795,0.940,1.025,1.025,0.825,...,-0.300,-0.290,-0.300,-0.295,-0.305,-0.285,-0.280,-0.285,-0.295,-0.300
342,S,-0.565,-0.550,-0.460,-0.340,-0.200,-0.050,0.170,0.440,0.675,...,-0.290,-0.290,-0.310,-0.295,-0.315,-0.310,-0.315,-0.305,-0.315,-0.325
441,S,-0.400,-0.385,-0.380,-0.370,-0.370,-0.410,-0.470,-0.490,-0.500,...,-0.240,-0.230,-0.225,-0.215,-0.215,-0.225,-0.205,-0.195,-0.170,-0.165
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108086,S,-0.200,-0.190,-0.200,-0.180,-0.150,-0.115,-0.030,0.055,0.170,...,-0.445,-0.450,-0.445,-0.440,-0.430,-0.435,-0.425,-0.420,-0.435,-0.445
108087,S,-0.155,-0.145,-0.145,-0.155,-0.150,-0.170,-0.165,-0.175,-0.185,...,-0.115,-0.095,-0.080,-0.070,-0.070,-0.075,-0.090,-0.100,-0.100,-0.085
108321,V,-0.360,-0.365,-0.360,-0.355,-0.350,-0.345,-0.350,-0.345,-0.335,...,-0.190,-0.210,-0.205,-0.210,-0.190,-0.185,-0.170,-0.160,-0.150,-0.160
108721,V,-0.145,-0.120,-0.110,-0.110,-0.115,-0.105,-0.095,-0.095,-0.085,...,-0.290,-0.285,-0.300,-0.310,-0.300,-0.290,-0.295,-0.310,-0.315,-0.300


In [21]:
value_counts = df_anormal["ColumnName"].value_counts()
print(value_counts)

ColumnName
Q    8043
V    7236
S    2781
F     803
Name: count, dtype: int64


In [22]:
X = df_anormal.iloc[:, 1:]
y = df_anormal.iloc[:, 0]

In [23]:
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout


label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded)

X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42)

# Train
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

/Users/arthurdesmazures/venv-metal/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [24]:
y_test

array([[0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       ...,
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.]], dtype=float32)

In [25]:
X_train.shape[1]

2160

In [26]:
y_train.shape[1]

4

In [27]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam

In [37]:
model = Sequential([
    Conv1D(filters=64, kernel_size=6, activation='relu', input_shape=(2160, 1)),
    MaxPooling1D(pool_size=3),
    BatchNormalization(),

    Conv1D(filters=128, kernel_size=6, activation='relu'),
    MaxPooling1D(pool_size=2),
    BatchNormalization(),

    Conv1D(filters=128, kernel_size=6, activation='relu'),
    MaxPooling1D(pool_size=2),
    BatchNormalization(),

    Conv1D(filters=64, kernel_size=6, activation='relu'),
    MaxPooling1D(pool_size=2),
    BatchNormalization(),

    Flatten(),

    Dense(64, activation='relu'),

    Dense(32, activation='relu'),

    Dense(4, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


# Summary of the model
model.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_15 (Conv1D)          (None, 2155, 64)          448       
                                                                 
 max_pooling1d_15 (MaxPooli  (None, 718, 64)           0         
 ng1D)                                                           
                                                                 
 batch_normalization_15 (Ba  (None, 718, 64)           256       
 tchNormalization)                                               
                                                                 
 conv1d_16 (Conv1D)          (None, 713, 128)          49280     
                                                                 
 max_pooling1d_16 (MaxPooli  (None, 356, 128)          0         
 ng1D)                                                           
                                                      

In [38]:
from tensorflow.keras.callbacks import EarlyStopping

# early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Example for the medium complexity model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_acc}')

Epoch 1/10
354/354 [==============================] - 14s 36ms/step - loss: 0.4498 - accuracy: 0.9100 - val_loss: 0.7126 - val_accuracy: 0.8031
Epoch 2/10
354/354 [==============================] - 12s 34ms/step - loss: 0.3175 - accuracy: 0.9443 - val_loss: 0.2777 - val_accuracy: 0.9465
Epoch 3/10
354/354 [==============================] - 12s 34ms/step - loss: 1.0670 - accuracy: 0.9250 - val_loss: 1.4973 - val_accuracy: 0.9335
Epoch 4/10
354/354 [==============================] - 12s 35ms/step - loss: 5.7875 - accuracy: 0.8975 - val_loss: 6.7188 - val_accuracy: 0.9316
Epoch 5/10
354/354 [==============================] - 12s 35ms/step - loss: 31.8966 - accuracy: 0.8734 - val_loss: 16.1506 - val_accuracy: 0.8741
Epoch 6/10
354/354 [==============================] - 12s 35ms/step - loss: 308.8495 - accuracy: 0.7129 - val_loss: 29.9282 - val_accuracy: 0.8301
Epoch 7/10
354/354 [==============================] - 12s 35ms/step - loss: 38.4793 - accuracy: 0.7992 - val_loss: 28.1875 - val_ac